In [ ]:
project = 'saga-nvdb-prod-vlmh'
use_colab_auth = False

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = 'saga-pedlan-playground-5b9x'

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen henter ut lengden på det nåværende vegnettet for riks- og europaveg i Norge.

Uttrekket ser på en forenklet utgave av europa- og riksvegnettet for kjørende, der adskilte felt og løp bare er gitt i én retning.
Dette gir en lengde for hovedårene i vegnettet men inkluderer da altså ikke lengdene for hvert kjørefelt, og heller ikke sideanlegg slik som gang- og sykkelveg.

In [ ]:
query = f"""
SELECT 
  SUM(ST_LENGTH(geometri))/1000 vegnettKm
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Nåværende vegnett
  (metadata.sluttdato IS NULL OR metadata.sluttdato >= CURRENT_DATE())

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen henter ut det nåværende vegnettet for riks- og europaveg i Trøndelag fylke.

Uttrekket viser en forenklet utgave av europa- og riksvegnettet for kjørende, der adskilte felt og løp bare er gitt i én retning.
Dette gir en oversikt over hvordan vegnettet overordnet ser ut, men unnlater en del detaljer, som feks hvor mange felt vegene har og hvor det finnes gang- og sykkelveg.

In [ ]:
query = f"""
SELECT 
  vegsystemreferanse.kortform, veglenkesekvensid, veglenkenummer, segmentnummer, startposisjon, sluttposisjon, typeVeg, detaljnivaa, kommune, geometri
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Trøndelag
  fylke = 50

  # Nåværende vegnett
  AND (metadata.sluttdato IS NULL OR metadata.sluttdato >= CURRENT_DATE())

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

vegnettTrondelag = client.query(query).to_dataframe()

vegnettTrondelag

Kartet under viser resultatet av forrige spørring. Merk at du kan åpne sidemenyen i kartet dersom du vil filtrere resultatet basert på feltverdier.

In [ ]:
from keplergl import KeplerGl

import geopandas as gpd
from shapely import wkt

vegnettTrondelagCopy = vegnettTrondelag.copy(deep=True)

vegnettTrondelagCopy['geometry'] = vegnettTrondelagCopy.geometri.apply(wkt.loads)
vegnettTrondelagCopy.drop('geometri', axis=1, inplace=True)

vegnettTrondelagGdf = gpd.GeoDataFrame(vegnettTrondelagCopy, geometry='geometry')

# Denne konfigen er laga ved å tweake kartet visuelt, og deretter hente ut map.config
mapConfig = {'version': 'v1',
              'config': {'visState': {'filters': [],
                'layers': [{'id': 'rtm9s4h',
                  'type': 'geojson',
                  'config': {'dataId': 'Europa- og riksveger i Trøndelag',
                    'label': 'Europa- og riksveger i Trøndelag',
                    'color': [231, 159, 213],
                    'highlightColor': [252, 242, 26, 255],
                    'columns': {'geojson': 'geometry'},
                    'isVisible': True,
                    'visConfig': {'opacity': 0.8,
                    'strokeOpacity': 0.8,
                    'thickness': 0.8,
                    'strokeColor': [246, 154, 0],
                    'colorRange': {'name': 'Global Warming',
                      'type': 'sequential',
                      'category': 'Uber',
                      'colors': ['#5A1846',
                      '#900C3F',
                      '#C70039',
                      '#E3611C',
                      '#F1920E',
                      '#FFC300']},
                    'strokeColorRange': {'name': 'Global Warming',
                      'type': 'sequential',
                      'category': 'Uber',
                      'colors': ['#5A1846',
                      '#900C3F',
                      '#C70039',
                      '#E3611C',
                      '#F1920E',
                      '#FFC300']},
                    'radius': 10,
                    'sizeRange': [0, 10],
                    'radiusRange': [0, 50],
                    'heightRange': [0, 500],
                    'elevationScale': 5,
                    'enableElevationZoomFactor': True,
                    'stroked': True,
                    'filled': False,
                    'enable3d': False,
                    'wireframe': False},
                    'hidden': False,
                    'textLabel': [{'field': None,
                      'color': [255, 255, 255],
                      'size': 18,
                      'offset': [0, 0],
                      'anchor': 'start',
                      'alignment': 'center'}]},
                  'visualChannels': {'colorField': None,
                    'colorScale': 'quantile',
                    'strokeColorField': None,
                    'strokeColorScale': 'quantile',
                    'sizeField': None,
                    'sizeScale': 'linear',
                    'heightField': None,
                    'heightScale': 'linear',
                    'radiusField': None,
                    'radiusScale': 'linear'}}],
                'interactionConfig': {'tooltip': {'fieldsToShow': {'Europa- og riksveger i Trøndelag': [{'name': 'kortform',
                      'format': None},
                    {'name': 'veglenkesekvensid', 'format': None},
                    {'name': 'veglenkenummer', 'format': None},
                    {'name': 'segmentnummer', 'format': None},
                    {'name': 'typeVeg', 'format': None},
                    {'name': 'detaljnivaa', 'format': None},
                    {'name': 'kommune', 'format': None},
                    {'name': 'startposisjon', 'format': None},
                    {'name': 'sluttposisjon', 'format': None}]},
                  'compareMode': False,
                  'compareType': 'absolute',
                  'enabled': True},
                  'brush': {'size': 0.5, 'enabled': False},
                  'geocoder': {'enabled': False},
                  'coordinate': {'enabled': False}},
                'layerBlending': 'normal',
                'splitMaps': [],
                'animationConfig': {'currentTime': None, 'speed': 1}},
                'mapState': {'bearing': 0,
                'dragRotate': False,
                'latitude': 63.76473477859502,
                'longitude': 10.603487695635067,
                'pitch': 0,
                'zoom': 5.8275970336811875,
                'isSplit': False},
                'mapStyle': {'styleType': 'light',
                'topLayerGroups': {},
                'visibleLayerGroups': {'label': True,
                  'road': True,
                  'border': True,
                  'building': True,
                  'water': True,
                  'land': True,
                  '3d building': False},
                'threeDBuildingColor': [218.82023004728686,
                  223.47597962276103,
                  223.47597962276103],
                'mapStyles': {}}}}

# Det er frivillig å sende med config, men nødvendig om ein vil kontrollere utsjånad
vegnettMap = KeplerGl(height=600, show_docs=False, config=mapConfig)
vegnettMap.add_data(data=vegnettTrondelagGdf, name="Europa- og riksveger i Trøndelag")
vegnettMap

Denne spørringen viser hvordan riks- og europavegnettet for et gitt tidspunkt kan hentes ut, med ellers samme filter som over.

In [ ]:
query = f"""
SELECT 
  vegsystemreferanse.kortform,
  geometri
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Trøndelag
  fylke = 50

  # Vegnett for et gitt tidspunkt
  AND (
      # Opprettet før det aktuelle tidspunktet 
      metadata.startdato < '2020-01-01'
      AND
      # Avsluttet etter tidspunktet eller er fortsatt aktiv
      (metadata.sluttdato >= '2020-01-01' OR metadata.sluttdato IS NULL)
  ) 

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen viser hvordan vegeier og vegens navn kan utledes fra de andre feltene.

In [ ]:
query = f"""
SELECT
  FORMAT("%s%s%d", 
    vegsystemreferanse.vegsystem.vegkategori, 
    vegsystemreferanse.vegsystem.fase, 
    vegsystemreferanse.vegsystem.nummer) vegnavn,
  IF(
    EXISTS(SELECT * FROM UNNEST(kontraktsomraader) WHERE REGEXP_CONTAINS(navn, "NV_")),
    "Stat, Nye Veier", 
    "Stat, Statens vegvesen") vegeier,
FROM 
  `{project}.standardized.veglenker`
WHERE
  # Trøndelag
  fylke = 50

  # Nåværende vegnett
  AND (metadata.sluttdato IS NULL OR metadata.sluttdato >= CURRENT_DATE())

  # Hovedtrase
  AND type = "HOVED"

  # Europaveg og Riksveg
  AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

  # Eksisterende veg som er del av det operative vegnettet
  AND vegsystemreferanse.vegsystem.fase in ("V")

  # Behold bare ett løp for adskilte tunelløp
  AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

  AND typeVeg IN (
    "Rundkjøring",
    "Enkel bilveg",
    "Kanalisert veg",
    "Rampe");
"""

print(query)

client.query(query).to_dataframe()


Denne spørringen viser hvordan man kan avdekke veglenkesekvenser som ble blitt opprettet i perioden 2020-01-01 til 2021-01-01. Merk at spørringen returnerer alle veglenker i sekvensen, ikke bare de som ble opprettet i den aktuelle perioden.

In [ ]:
query = f"""
WITH 
veglenker AS 
  (SELECT *
  FROM `{project}.standardized.veglenker`
  WHERE 
    # Hovedtrase
    type = "HOVED"

    # Europaveg og Riksveg
    AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

    # Eksisterende veg som er del av det operative vegnettet
    AND vegsystemreferanse.vegsystem.fase in ("V")

    # Behold bare ett løp for adskilte tunelløp
    AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

    AND typeVeg IN (
      "Rundkjøring",
      "Enkel bilveg",
      "Kanalisert veg",
      "Rampe")),

nye_veglenkesekvenser AS
  (SELECT
    veglenkesekvensid,
    ST_UNION_AGG(geometri) geometri,
    MIN(metadata.startdato) AS tidligste_startdato
  FROM veglenker
  GROUP BY veglenkesekvensid
  HAVING
   # Sekvensen ble opprettet i perioden
   tidligste_startdato BETWEEN '2020-01-01' AND '2021-01-01')

SELECT * FROM nye_veglenkesekvenser
"""

print(query)

client.query(query).to_dataframe()

Denne spørringen viser hvordan man kan avdekke veglenkesekvenser som ble endelig avsluttet i perioden 2020-01-01 til 2021-01-01. Dette er en grei måte å få oversikt over større endringer i vegnettet, men den får ikke med seg de tilfeller hvor det blir avsluttet noen, men ikke alle, veglenker i en sekvens. [Se denne presentasjonen for mer informasjon om forholdet mellom veglenker og veglenkesekvenser.](https://vegvesen-my.sharepoint.com/:p:/g/personal/jan_kristian_jensen_vegvesen_no/ERTPmjdinZxIh23EwpQeigYBn_lFAzyJHuuisQyU4_qVhg)

Merk at spørringen returnerer alle veglenker i sekvensen, ikke bare de som ble avsluttet i den aktuelle perioden.

In [ ]:

query = f"""
WITH 
veglenker AS 
  (SELECT *
  FROM `{project}.standardized.veglenker`
  WHERE 
    # Hovedtrase
    type = "HOVED"

    # Europaveg og Riksveg
    AND vegsystemreferanse.vegsystem.vegkategori in ("E", "R")

    # Eksisterende veg som er del av det operative vegnettet
    AND vegsystemreferanse.vegsystem.fase in ("V")

    # Behold bare ett løp for adskilte tunelløp
    AND (vegsystemreferanse.strekning.adskilteLop IN ("Med", "Nei") OR vegsystemreferanse.strekning.adskilteLop IS NULL)

    AND typeVeg IN (
      "Rundkjøring",
      "Enkel bilveg",
      "Kanalisert veg",
      "Rampe")),

stengte_veglenkesekvenser AS
  (SELECT
    veglenkesekvensid,
    ST_UNION_AGG(geometri) geometri,
    COUNT(*) AS totalt_antall_veglenker,
    SUM(
      CASE
        WHEN metadata.sluttdato IS NOT NULL THEN 1
        ELSE 0
      END) antall_stengte_veglenker,
    MAX(metadata.sluttdato) AS siste_sluttdato
  FROM veglenker
  GROUP BY veglenkesekvensid
  HAVING
   # alle veglenkene i veglenkesekvensen er stengt
   totalt_antall_veglenker = antall_stengte_veglenker
   # siste sluttdato i perioden
   AND siste_sluttdato BETWEEN '2020-01-01' AND '2021-01-01')

SELECT * FROM stengte_veglenkesekvenser
"""

print(query)

client.query(query).to_dataframe()